# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [5]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

#!pip install ollama -qq
import ollama



In [14]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = "llama3.2:1b"

In [7]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    

openai = OpenAI()

API key looks good so far


In [ ]:
# Get gpt-4o-mini to answer, with streaming

def open_ai_tutor(python_question):
    system_prompt = """
    Imagine you are a python tutor and you have to explain what the following code does. 
    You are teaching a course in intermediate python. Help the student understand the following question.
    Respond in Markdown.
    """
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": python_question}
          ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

open_ai_tutor(question)

Certainly! Let's break down the code snippet you've provided:

python
yield from {book.get("author") for book in books if book.get("author")}


## Explanation

### 1. Set Comprehension
The code uses a set comprehension, which is a concise way to create a set in Python. The syntax for a set comprehension looks like this:

python
{expression for item in iterable if condition}


In this case:
- `book.get("author")` is the expression that extracts the author from each `book` in `books`.
- `for book in books` iterates through a collection called `books`, which is presumably a list (or another iterable) of dictionaries.
- `if book.get("author")` is a condition that filters out any `book` that does not have an author or has an empty author. This ensures that only valid authors are included in the set.

### 2. `yield from`
The `yield from` statement is used in a generator function. It allows the generator to yield all values from another iterable (in this case, the set created by the set comprehension). This is useful when you want to yield values from a nested generator or from another iterable without needing a loop to handle that explicitly.

### 3. Resulting Set
The overall effect is that this piece of code generates a set of unique authors from the `books` iterable, ignoring any entries that do not have an author. This construct automatically handles duplicates because sets only store unique values.

### 4. Why Use It
- **Efficiency in Uniqueness**: By using a set comprehension, the code succinctly filters out duplicates and directly creates a set of authors.
- **Readability**: The use of `yield from` makes the generator simple to write and maintain, as it abstracts away the looping logic that would otherwise be needed to yield each author.
  
## Example

Suppose you have the following `books` list:

python
books = [
    {"title": "Book A", "author": "Author X"},
    {"title": "Book B", "author": "Author Y"},
    {"title": "Book C", "author": "Author X"},
    {"title": "Book D", "author": None},
]


The set comprehension `{book.get("author") for book in books if book.get("author")}` would evaluate to:

python
{"Author X", "Author Y"}


Then, the `yield from` statement would yield each author in this set, making it possible for the calling code to process each author one at a time without having to collect them all into a list first.

## Conclusion

The code you provided is an efficient and pythonic way to extract unique authors from a list of dictionaries, while also handling missing or empty authors neatly.

In [20]:
# Get Llama 3.2 to answer

def ollama_tutor(python_question):
    system_prompt = """
    Imagine you are a python tutor and you have to explain what the following code does. 
    You are teaching a course in intermediate python. Help the student understand the following question.
    Respond in Markdown.
    """
    response = ollama.chat(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": python_question}
          ],
    )
    print(display(Markdown(response['message']['content']), display_id=True))

ollama_tutor(question)


## Explanation of the Code

The given code is a part of a Python loop that uses the `yield` keyword, which is used to generate an iterator.

```python
yield from {book.get("author") for book in books if book.get("author")}
```

This line of code is using a technique called "generator expression" or "map-like iterable". It's a concise way to create a generator that yields values on-the-fly, instead of computing them all at once and storing them in memory.

Let's break down the components:

- `yield from`: This keyword is used to convert an iterable (like a list or dictionary) into an iterator. The result is another iterator.
- `{book.get("author") for book in books if book.get("author")}`: This part is generating a sequence of values, one at a time, using a generator expression.

Here's how it works:

1. `books` is assumed to be a list or other iterable data structure that you want to iterate over.
2. The generator expression inside the outer loop (`{book.get("author") for book in books if book.get("author")}`) generates a sequence of values, one at a time.

The outer loop iterates over each `book` in `books`, and checks if its "author" key exists using `book.get("author")`. If it does, the value is yielded (i.e., returned) to the outer loop's iterator.

- `yield`: This keyword is used to define a generator function. When executed, it returns an iterator that produces values on-the-fly.
- The entire expression becomes an iterator because of the final `yield from` statement.

## Example Use Case

Suppose you have a list of dictionaries where each dictionary represents a book:

```python
books = [
    {"author": "John"},
    {"author": "Jane"},
    {"author": None}
]
```

If we call `books.get("author")`, it will return `"John"` and `"Jane"`. However, if we want to generate a list of authors without loading all books into memory at once:

```python
for author in yield from {book.get("author") for book in books if book.get("author")}:
    print(author)
```

This code will iterate over the authors (`John` and `Jane`) one by one, printing each value to the console.

## Code Quality Notes

- The use of `yield from` here is a bit unusual because it doesn't actually create an iterator that you can call again. Instead, it converts the iterable into an iterator on-the-fly.
- It's worth noting that this syntax only works for iterables (like lists or dictionaries). If you try to use it with other types of data structures like tuples or sets, it will not work as expected.

## Best Practices

- In general, when using generators in Python, consider whether you need the iterator on-the-fly. They can be slower than loading all values at once.
- When converting an iterable into an iterator, always use `yield from` instead of directly applying it to a variable (e.g., `... for book in books`).

<DisplayHandle display_id=32679e2173670dbf0140dd0dc768e970>
